# Sighash modifiers - BX Exercise
In this exercise, we spend two p2pkh inputs, signing with sighash modifiers `single|anyonecanpay` and `none`

<hr style="border: 0.5px dashed #000;">

## 1. Restore wallet and generate spending + receiving key pairs.



In [ ]:
# my_mnemonic ="word0 word1 word2 ..."
# hd_master_private=$( )
# hd_master_public=$( )

my_mnemonic="amount right cheese defy click eight slight strategy replace earn simple labor"
hd_master_private=$(bx mnemonic-to-seed $my_mnemonic | bx hd-new)
hd_master_public=$(bx hd-to-public $hd_master_private)

### Restore your spending keys.

We will be spending the funds controlled by these keys from account 1. 
* `m/44'/1'/1'/0/1`
* `m/44'/1'/1'/0/2`.


In [ ]:
# hd_m_44h_1h_1h_0_1=

# privatekey_44h_1h_1h_0_1=
# publickey_44h_1h_1h_0_1=
# publickeyhash_44h_1h_1h_0_1=
# address_44h_1h_1h_0_1=

hd_m_44h_1h_1h_0_1=$(bx hd-private --hard --index 44 $hd_master_private \
| bx hd-private --hard --index 1  \
| bx hd-private --hard --index 1  \
| bx hd-private --index 0 \
| bx hd-private --index 1)

privatekey_44h_1h_1h_0_1=$(bx hd-to-ec $hd_m_44h_1h_1h_0_1)
publickey_44h_1h_1h_0_1=$(bx hd-to-public $hd_m_44h_1h_1h_0_1 | bx hd-to-ec)
publickeyhash_44h_1h_1h_0_1=$(bx sha256 $publickey_44h_1h_1h_0_1 | bx ripemd160)
address_44h_1h_1h_0_1=$(bx hd-to-public $hd_m_44h_1h_1h_0_1 | bx hd-to-ec | bx ec-to-address --version 111)

echo $address_44h_1h_1h_0_1
echo $publickeyhash_44h_1h_1h_0_1

In [ ]:
# hd_m_44h_1h_1h_0_2=

# privatekey_44h_1h_1h_0_2=
# publickey_44h_1h_1h_0_2=
# publickeyhash_44h_1h_1h_0_2=
# address_44h_1h_1h_0_2=

hd_m_44h_1h_1h_0_2=$(bx hd-private --hard --index 44 $hd_master_private \
| bx hd-private --hard --index 1  \
| bx hd-private --hard --index 1  \
| bx hd-private --index 0 \
| bx hd-private --index 2)

privatekey_44h_1h_1h_0_2=$(bx hd-to-ec $hd_m_44h_1h_1h_0_2)
publickey_44h_1h_1h_0_2=$(bx hd-to-public $hd_m_44h_1h_1h_0_2 | bx hd-to-ec)
publickeyhash_44h_1h_1h_0_2=$(bx sha256 $publickey_44h_1h_1h_0_2 | bx ripemd160)
address_44h_1h_1h_0_2=$(bx hd-to-public $hd_m_44h_1h_1h_0_2 | bx hd-to-ec | bx ec-to-address --version 111)

echo $address_44h_1h_1h_0_2
echo $publickeyhash_44h_1h_1h_0_2

### Fetch previous UTXO's you are spending.

In [ ]:
# bx fetch-tx [transaction_hash] -f json | jq ".transaction.outputs[output_index_0]"
# bx fetch-tx [transaction_hash] -f json | jq ".transaction.outputs[output_index_1]"

bx fetch-tx 86ad635645920497c233cc33556463fa6258ab8f9c5f7d12748a6638216e3363 -f json | jq ".transaction.outputs[1]"
bx fetch-tx 86ad635645920497c233cc33556463fa6258ab8f9c5f7d12748a6638216e3363 -f json | jq ".transaction.outputs[2]"

In [ ]:
# previous_txid0=
# previous_output_index0=
# previous_output_amount0=

# previous_txid1=
# previous_output_index1=
# previous_output_amount1=

previous_txid0=86ad635645920497c233cc33556463fa6258ab8f9c5f7d12748a6638216e3363
previous_output_index0=1
previous_output_amount0=499543

previous_txid1=86ad635645920497c233cc33556463fa6258ab8f9c5f7d12748a6638216e3363
previous_output_index1=2
previous_output_amount1=499543

## 2. Sign first input and output with `single|anyonecanpay`

### Create first  P2PKH output script.

* Please send testnet coins to the following address: `n2MBcctgzBt1h8Nvfu3XAEPJLrmWET7emw`

In [ ]:
# bx decode-address n2MBcctgzBt1h8Nvfu3XAEPJLrmWET7emw
bx address-decode n2MBcctgzBt1h8Nvfu3XAEPJLrmWET7emw

In [ ]:
# bx script-encode "DUP HASH160 [public key hash] EQUALVERIFY CHECKSIG"

# output_script0=

output_script0=$(bx script-encode "DUP HASH160 ["$e48199d47742b245464b1366d95ef26aa4c8bb2c"] EQUALVERIFY CHECKSIG")
echo $output_script

### Set output amount equal to UTXO value.
* Fees will be later be deducted from output at index 1.

In [ ]:
# Output amount total.
output_amount0=$previous_output_amount0
echo $output_amount0


### Construct transaction template for signing input and outputs at index 0.

* `sequence: 0xffffffff(hex)/4294967295(dec)`

In [ ]:
# bx tx-encode \
# --input [previous tx id]:[index]:[sequence] \
# --output [output script]:[output amount]

my_tx=$(bx tx-encode \
--input $previous_txid0:$previous_output_index0:4294967295)

In [ ]:
# bx fetch-tx --format json [previous tx id] 
previous_output_script=$(bx fetch-tx --format json $previous_txid0 \
| jq -r ".transaction.outputs[1].script")

echo $previous_output_script